In [2]:
#Load modules
import os
import pandas as pd
import numpy as np

In [3]:
#Set filenames
dataDir = '../../Data'
usageFN = dataDir + os.sep + "AllUsageData.csv"
supplyFN = dataDir + os.sep + "SupplyData.csv"
stateFIPSFn = dataDir + os.sep + "FIPSstate.csv"
outFN = dataDir + os.sep + "UseAndSupply.csv"

In [16]:
#Create data frames from usage and supply
dfUsage = pd.read_csv(usageFN,dtype={'FIPS':np.str})
dfSupply = pd.read_csv(supplyFN,dtype={'FIPS':np.str})

In [17]:
#Add State FIPS codes columns
dfUsage['STFIPS'] = dfUsage['FIPS'].apply(lambda x: str(x)[:2] )
dfSupply['STFIPS'] = dfSupply['FIPS'].apply(lambda x: str(x)[:2] )

In [18]:
#Load state FIPS lookup table and join state codes
dfFIPS = pd.read_csv(stateFIPSFn,dtype=np.str)
dfSupply = pd.merge(dfSupply,dfFIPS,how='left',left_on='STFIPS',right_on='state_fips')
dfUsage = pd.merge(dfUsage,dfFIPS,how='left',left_on='STFIPS',right_on='state_fips')

In [19]:
#Compute supply as precip - et
dfSupply['SUPPLY'] = dfSupply['pr'] - dfSupply['et']

In [ ]:
#Drop the FIPS from the tables
dfUsage.drop(['STFIPS','state_fips','FIPS'], axis=1, inplace = True)
dfSupply.drop(['STFIPS','state_fips','FIPS','total_runoff','pr','et','smc'], axis=1, inplace = True)

In [ ]:
#Group the supply table on state
grpSupply = dfSupply.groupby(['YEAR','state']).sum()

In [ ]:
#Group the usage table on state
grpUsage = dfUsage.groupby(['YEAR','state']).sum()

In [ ]:
#join tables on year and fips
dfX = pd.merge(grpSupply, grpUsage, how='outer',left_index=True,right_index=True)
dfX.to_csv(outFN,na_rep=0)

In [25]:
dfUsage.head()

,YEAR,FIPS,Public,Domestic,Industry,Irrigation,Livestock,Aquaculture,Mining,Thermoelectic,TOTAL,STFIPS,state_fips,state
0,2000,01001,5.70,2.95,28.29,NaN,NaN,0.0,NaN,0.0,37.25,01,01,AL
1,2000,01003,19.70,2.46,6.49,NaN,NaN,0.0,NaN,0.0,34.63,01,01,AL
2,2000,01005,4.85,0.51,0.94,NaN,NaN,0.0,NaN,0.0,7.42,01,01,AL
3,2000,01007,3.78,0.22,0.00,NaN,NaN,0.0,NaN,0.0,4.00,01,01,AL
4,2000,01009,31.81,0.38,0.00,NaN,NaN,0.0,NaN,0.0,32.26,01,01,AL
